In [6]:
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
# from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.circuit.library import HartreeFock
from qiskit.circuit.library import TwoLocal
from qiskit import Aer,transpile, QuantumCircuit
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms import VQE
from IPython.display import display, clear_output
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 
from qiskit_nature.transformers import FreezeCoreTransformer  

In [7]:
molecule = "Li 0.0 0.0 0.0; H 0.0 0.0 1.5474"
driver = PySCFDriver(atom=molecule, basis="sto3g")
qmolecule = driver.run()

In [8]:
backend = Aer.get_backend('statevector_simulator')
# Remove the orbitals which do not interact much 
# Refer to https://www.youtube.com/watch?v=3B04KB0pDwE for removing orbitals
freeze_core_transformer = FreezeCoreTransformer(freeze_core=True,remove_orbitals=[3,4])
problem = ElectronicStructureProblem(driver,q_molecule_transformers=[freeze_core_transformer]) 
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0]
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals

# Generate the second-quantized operators
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0]

mapper = ParityMapper()
# z2symmetry_reduction is not required as such as we have already reduced our ciruit to 4 qubits
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True,z2symmetry_reduction='auto')
qubit_op = converter.convert(main_op, num_particles=num_particles,sector_locator=problem.symmetry_sector_locator)
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

# There is no exact science to Ansatz. So this is more of a trial and error with CNOTS. Use iterator across  entangler_map to get the best CNOTs
rotation_blocks = ['ry']
entanglement_blocks = 'cx'
entangler_map = [[[3, 0]], [[3, 1]], [[3, 2]]]
ansatz = TwoLocal(qubit_op.num_qubits,
                  rotation_blocks,
                  entanglement_blocks,
                  reps=3, 
                  entanglement=entangler_map,
                  skip_final_rotation_layer=True,
                  skip_unentangled_qubits=True,
                  insert_barriers=True)


ansatz.compose(init_state, front=True, inplace=True)
display(ansatz.draw('text'))

initial_point = [0.01] * ansatz.num_parameters
optimizer = SLSQP(maxiter=500)
algorithm = VQE(ansatz,optimizer=optimizer,quantum_instance=backend,initial_point=initial_point)
result = algorithm.compute_minimum_eigenvalue(qubit_op)

┌───┐    ┌──────────┐ ░ ┌───┐ ░ ┌──────────┐ ░       ░  ┌──────────┐ ░ »
q_0: ───┤ X ├────┤ RY(θ[0]) ├─░─┤ X ├─░─┤ RY(θ[4]) ├─░───────░──┤ RY(θ[8]) ├─░─»
        ├───┤    ├──────────┤ ░ └─┬─┘ ░ ├──────────┤ ░ ┌───┐ ░  ├──────────┤ ░ »
q_1: ───┤ X ├────┤ RY(θ[1]) ├─░───┼───░─┤ RY(θ[5]) ├─░─┤ X ├─░──┤ RY(θ[9]) ├─░─»
     ┌──┴───┴───┐└──────────┘ ░   │   ░ ├──────────┤ ░ └─┬─┘ ░ ┌┴──────────┤ ░ »
q_2: ┤ RY(θ[2]) ├─────────────░───┼───░─┤ RY(θ[6]) ├─░───┼───░─┤ RY(θ[10]) ├─░─»
     ├──────────┤             ░   │   ░ ├──────────┤ ░   │   ░ ├───────────┤ ░ »
q_3: ┤ RY(θ[3]) ├─────────────░───■───░─┤ RY(θ[7]) ├─░───■───░─┤ RY(θ[11]) ├─░─»
     └──────────┘             ░       ░ └──────────┘ ░       ░ └───────────┘ ░ »
«          
«q_0: ─────
«          
«q_1: ─────
«     ┌───┐
«q_2: ┤ X ├
«     └─┬─┘
«q_3: ──■──
«

In [9]:
solver = NumPyMinimumEigensolverFactory()
calc = GroundStateEigensolver(converter, solver)
energy = result.optimal_value
result_exact = calc.solve(problem)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Energy", energy)
print("Exact energy", exact_energy)
print("Error", (energy-exact_energy)*1000)
# print("Delta",(energy-exact_energy)*1000 - 4)
# print("Pass",(energy-exact_energy)*1000 <= 4)
print("Number of parameters",ansatz.num_parameters)

Energy -1.0863580501410701
Exact energy -1.0887060157347417
Error 2.3479655936715638
Number of parameters 12


In [10]:
from qc_grader import grade_ex5
freeze_core = True # change to True if you freezed core electrons
grade_ex5(ansatz,qubit_op,result,freeze_core)

Grading your answer for ex5. Please wait...
/Users/rj/.virtualenvs/qiskit/lib/python3.9/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(

Congratulations 🎉! Your answer is correct.
Your cost is 3.
Feel free to submit your answer.

